In [ ]:
from appgeopy import *
from my_packages import *

In [ ]:
pkl_fpath = r"E:\030_CHOUSHUI_2024\000_INSCALDEFO_2_INSSTACKPSI\PROCESS_003\ras2pnt_oChoushui_CUMDISP_LOS_mm.pkl"
cumdisp_los_df = pd.read_pickle(pkl_fpath, compression="zip")
cumdisp_los_df.head(5)

In [ ]:
processing_year = 2023
datetime_colnames = pd.Series(
    ["N20220326"] + [char for char in cumdisp_los_df if char.startswith("N")]
)
cumdisp_los_datetime = datetime_colnames.apply(
    datetime_handle.convert_to_datetime
)

"""
_temp = cumdisp_los_datetime[
    cumdisp_los_datetime.dt.year == processing_year
]

current_first_index = _temp.index[0]
current_last_index = _temp.index[-1]

if current_first_index!=0:
    target_datetime = cumdisp_los_datetime.iloc[(current_first_index-1):current_last_index+1]
else:
    target_datetime = _temp
"""

target_datetime = cumdisp_los_datetime[
    cumdisp_los_datetime.dt.year == processing_year
]

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

mainfolder = r"E:\SUBSIDENCE_PROJECT_DATA\GPS_2023_葉 大綱\2023_NEW_20240515\20240527_PROCESSED_DATA\_{}".format(
    processing_year
)

mod_folder = os.path.join(mainfolder, "1__Datetime_Modified")
mod_files = glob(os.path.join(mod_folder, "*.xlsx"))
mod_files[:5], len(mod_files)

In [ ]:
target_datetime[:5], target_datetime[-5:]

In [ ]:
output_savefolder = r"E:\SUBSIDENCE_PROJECT_DATA\GPS_2023_葉 大綱\2023_NEW_20240515\20240528_INSAR_GPS_INTERSECT"
output_excel_filename = "InSAR_GPS_Intersect_{}.xlsx".format(processing_year)
output_excel_filepath = os.path.join(output_savefolder, output_excel_filename)

cache = {"STATION": [], "Full_Length": [], "File_Length": []}

# select_file = mod_files[0]
for select_file in mod_files:
    station_name = os.path.basename(select_file).split("_")[0]

    gps_fulltime_df = pd.read_excel(select_file, parse_dates=[0], index_col=[0])

    gps_select_data = gps_fulltime_df.loc[target_datetime, :]

    notnull_data = gps_select_data.notnull().sum()["h(cm)"]

    for ele, key in zip(
        [station_name, len(target_datetime), notnull_data], cache.keys()
    ):
        cache[key].append(ele)

    data_io.save_df_to_excel(
        df_to_save=gps_select_data,
        filepath=output_excel_filepath,
        sheet_name=station_name,
        mode="a",
        index=True,
        verbose=False,
    )

In [ ]:
pd.DataFrame(cache).to_excel(
    r"E:\SUBSIDENCE_PROJECT_DATA\GPS_2023_葉 大綱\2023_NEW_20240515\20240528_INSAR_GPS_INTERSECT\Summary_{}.xlsx".format(
        processing_year
    ),
    index=False,
)